In [42]:
import pandas as pd
import numpy as np
#import geopandas as gp
#import adjustText as aT
#import matplotlib.pylab as plt
from sklearn import neighbors, datasets
#from matplotlib.colors import ListedColormap

#df_train = pd.read_csv('data/train_filtrado.csv')
#df_test = pd.read_csv('data/test.csv')

In [43]:
# maps = gp.read_file('data/maps/dest_2010gw.shp')

In [44]:
# vals = df_train[(~df_train.lat.isnull()) & (~df_train.lng.isnull()) & (df_train.lat < 32.769750) & (df_train.lat > 14) & (df_train.lng > -118) & (df_train.lng < -85)]
# ax = maps.plot(color='white', edgecolor='black')
# vals.plot.scatter('lng', 'lat', ax=ax, c='precio', colormap='jet', figsize=(30, 15), colorbar=False,
#                 title='Precio de propiedad y su ubicación en el mapa - México');

# sm = plt.cm.ScalarMappable(cmap='jet', 
#                            norm=plt.Normalize(vmin=vals.precio.min(),
#                                               vmax=vals.precio.max()))
# plt.colorbar(sm, label='precio');

In [45]:
# vals.shape

In [46]:
def merge_ciudades(df):
    ciudades_centricas = pd.read_csv('./data/ciudadcentrica_por_provincia.csv')
    df_centro = pd.merge(df, ciudades_centricas, on = 'ciudad', how= 'right').rename(columns={'provincia_x': 'provincia'})
    df_centro['centro'] =  df_centro['centro'].fillna(False)
    df_centro_promedio = df_centro.groupby('provincia')[['lat', 'lng']].agg('mean').rename(columns = {'lat': 'lat_promedio', 'lng': 'lng_promedio'}).reset_index()
    lat_DF = float(df_centro_promedio[df_centro_promedio['provincia'] == 'Distrito Federal']['lat_promedio'])
    lng_DF = float(df_centro_promedio[df_centro_promedio['provincia'] == 'Distrito Federal']['lng_promedio'])

    return df_centro_promedio, lat_DF, lng_DF

In [47]:
def merge_ciudades_costosas(df):
    ciudades_costosas = pd.read_csv('./data/ciudad_costosa.csv')
    df_cara = pd.merge(df, ciudades_costosas, on = 'ciudad', how= 'right').rename(columns={'provincia_x': 'provincia'})
    df_cara['costosa'] =  df_cara['costosa'].fillna(False)
    df_costo_promedio = df_cara.groupby('provincia')[['lat', 'lng']].agg('mean').rename(columns = {'lat': 'lat_promedio', 'lng': 'lng_promedio'}).reset_index()
    return df_costo_promedio

In [48]:
#Verifico que todo se encuentre en rango:
#df_centro_promedio[(~df_centro_promedio.lat_promedio.isnull()) & (~df_centro_promedio.lng_promedio.isnull()) & (df_centro_promedio.lat_promedio < 32.769750) & (df_centro_promedio.lat_promedio > 14) & (df_centro_promedio.lng_promedio > -118) & (df_centro_promedio.lng_promedio < -85)].shape

In [49]:
def feature_distancias(df, df_fill=None):
    if df_fill is None:
        df_fill = df
    
    
    df_centro_promedio, lat_DF, lng_DF = merge_ciudades(df_fill)
    df_costo_promedio = merge_ciudades_costosas(df_fill)
    
    df_merge = pd.merge(df, df_centro_promedio, on = 'provincia', how= 'left')
    df['distancia_ciudad_centrica'] = np.sqrt((df_merge.lat - df_merge.lat_promedio)**2 + (df_merge.lng -df_merge.lng_promedio)**2)
    df['distancia_ciudad_centrica'] = df['distancia_ciudad_centrica'].fillna(df['distancia_ciudad_centrica'].mean())    
    
    
    df['distancia_centro_mexico'] = np.sqrt((df_merge.lat - lat_DF)**2 + (df_merge.lng - lng_DF)**2)
    df['distancia_centro_mexico'] = df['distancia_centro_mexico'].fillna(df['distancia_centro_mexico'].mean())    
    
    
    df_merge_costo = pd.merge(df,df_costo_promedio, on= 'provincia', how = 'left' )
    df['distancia_ciudad_cara'] = np.sqrt((df_merge_costo.lat - df_merge_costo.lat_promedio)**2 + (df_merge_costo.lng -df_merge_costo.lng_promedio)**2)
    df['distancia_ciudad_cara'] = df['distancia_ciudad_cara'].fillna(df['distancia_ciudad_cara'].mean())    
    
    return df